This file deploys the perfume recommender on Streamlit, including data with categorized reviews into sentiments, **tag generation**, Sentence-BERT model and LLM for perfume explanation.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install streamlit unsloth sentence-transformers

In [ ]:
# To view Streamlit app in a browser later
!pip install pyngrok

In [ ]:
%%writefile streamlit_app.py

import streamlit as st
import torch
import re
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


st.title("Perfume Recommendation with Tag Generation")
st.write("🌀 Starting app...")

@st.cache_resource
def load_tag_model():
    quantized_model_path = "/content/drive/MyDrive/Colab Notebooks/totallymakescents/perfume_mistral_cpt_fine_tune_adapters-4bit"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )

    tag_model = AutoModelForCausalLM.from_pretrained(
        quantized_model_path,
        device_map="auto",
        quantization_config=bnb_config
    )
    tag_tokenizer = AutoTokenizer.from_pretrained(quantized_model_path)

    return tag_model, tag_tokenizer


@st.cache_resource
def load_model():
    save_path = "/content/drive/MyDrive/Colab Notebooks/totallymakescents/llama-model/"
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=save_path,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
    FastLanguageModel.for_inference(model)
    return model, tokenizer

@st.cache_resource
def load_sbert():
    return SentenceTransformer("all-MiniLM-L6-v2")

@st.cache_resource
def load_embeddings():
    return torch.load("/content/drive/MyDrive/Colab Notebooks/totallymakescents/perfume_embeddings.pt")

@st.cache_resource
def load_dataframe():
    return pd.read_csv("/content/drive/MyDrive/Colab Notebooks/totallymakescents/data/combined_df_classify_reviews.csv")

try:
    st.write("🔧 Loading LLM tag generation model...")
    tag_model, tag_tokenizer = load_tag_model()
    st.success("✅ Tag model loaded!")
except Exception as e:
    st.error(f"❌ Error loading model: {e}")
    st.stop()

try:
    st.write("🔧 Loading LLM explanation model...")
    model, tokenizer = load_model()
    st.success("✅ Explanation model loaded!")
except Exception as e:
    st.error(f"❌ Error loading model: {e}")
    st.stop()

try:
    st.write("📔 Loading SBERT...")
    sbert_model = load_sbert()
    st.success("✅ SBERT loaded!")
except Exception as e:
    st.error(f"❌ Error loading SBERT: {e}")
    st.stop()

try:
    st.write("💐 Loading perfume embeddings...")
    perfume_embeddings = load_embeddings()
    st.success("✅ Embeddings loaded!")
except Exception as e:
    st.error(f"❌ Error loading embeddings: {e}")
    st.stop()

try:
    st.write("📄 Loading data...")
    combined_df_classify_reviews = load_dataframe()
    st.success("✅ Data loaded!")
except Exception as e:
    st.error(f"❌ Error loading data: {e}")
    st.stop()

#generate tags
def generate_tags(prompt, max_new_tokens=128):
    tag_model.eval()
    input_text = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tag_tokenizer(input_text, return_tensors="pt").to(tag_model.device)

    with torch.no_grad():
        output = tag_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tag_tokenizer.decode(output[0], skip_special_tokens=True)
    cleaned = decoded.replace("<|im_start|>", "").replace("<|im_end|>", "").strip()
    cleaned = re.sub(r"\b(nassistant\n|assistant\n|user\n):?", "", cleaned, flags=re.IGNORECASE)

    if "### Response:" in cleaned:
        cleaned = cleaned.split("### Response:")[-1].strip()

    return cleaned


#input format for LLM
def format_for_explanation_with_tags(user_query, perfume_row, tags):
    short_desc = (
        f"Top Notes: {perfume_row['Top']}. "
        f"Middle Notes: {perfume_row['Middle']}. "
        f"Base Notes: {perfume_row['Base']}. "
        f"Main Accords: {', '.join([str(perfume_row.get(f'mainaccord{i}', '')) for i in range(1, 6)])}."
    )
    return {
        "role": "user",
        "content": (
            f"User query: {user_query}\n"
            f"Perfume returned: {perfume_row['Perfume']} by {perfume_row['Brand']}\n"
            f"Notes: {short_desc}\n"
            f"Tags: {tags}\n"
            f"Please explain why this perfume fits the request, using both the tags and the notes."
        )
    }

#builds the app
user_query = st.text_input("Describe your scent preference:", placeholder="e.g. Looking for a bittersweet scent for a farewell party.")
top_k = st.slider("Number of Recommendations", min_value=1, max_value=5, value=3)

if st.button("Recommend and Explain") and user_query:
    with st.spinner("Finding matches and generating explanations..."):
      tags_text = generate_tags(user_query)

      query_embedding = sbert_model.encode(tags_text, convert_to_tensor=True)
      scent_tensor = perfume_embeddings.to(query_embedding.device)

      similarities = util.cos_sim(query_embedding, scent_tensor)[0]

      adjusted_scores = [] # boost scores of positive perfumes
      for idx, score in enumerate(similarities):
          sentiment_boost = 1.2 if combined_df_classify_reviews.loc[idx, 'is_positive'] == 1 else 0.8
          adjusted_scores.append(score.item() * sentiment_boost)

      adjusted_scores = torch.tensor(adjusted_scores)
      top_results = torch.topk(similarities, k=top_k)

      for score, idx in zip(top_results.values, top_results.indices):
          idx = idx.item() # convert Python tensor to int
          perfume = combined_df_classify_reviews.iloc[idx]
          message = format_for_explanation_with_tags(user_query, perfume, tags_text)

          inputs = tokenizer.apply_chat_template(
              [message],
              tokenize=True,
              add_generation_prompt=True,
              return_tensors="pt",
          ).to("cuda")

          text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
          with st.expander(f"{perfume['Perfume']} by {perfume['Brand']} (Score: {score.item():.3f})", expanded=True):
              short_desc = (
                  f"Top Notes: {perfume['Top']}. "
                  f"Middle Notes: {perfume['Middle']}. "
                  f"Base Notes: {perfume['Base']}. "
                  f"Main Accords: {', '.join([str(perfume.get(f'mainaccord{i}', '')) for i in range(1, 6)])}."
              )

              st.markdown(f"**Notes**: {short_desc}")

              # LLM Explanation
              output_llm = model.generate(
                    input_ids=inputs,
                    max_new_tokens=256,
                    use_cache=True,
                    temperature=1.5,
                    min_p=0.1,
                    streamer=text_streamer
                )

              full_output_llm = tokenizer.decode(output_llm[0], skip_special_tokens=True)

              assistant_prefix = "assistant\n"
              if assistant_prefix in full_output_llm:
                  llm_explanation = full_output_llm.split(assistant_prefix, 1)[-1].strip()
              else:
                  llm_explanation = full_output_llm.replace(message["content"], "").strip()

              st.markdown("**Explanation:**")
              st.markdown(llm_explanation)

Overwriting streamlit_app.py


In [ ]:
from google.colab import userdata
from pyngrok import conf, ngrok
ngrok.kill()  # reset tunnels

ngrok_token = userdata.get('ngrok_KEY') # needs key from ngrok

conf.get_default().auth_token = ngrok_token

public_url = ngrok.connect(addr=8501, proto="http")
print("Visit the app in the first link, not the local link:\n", public_url)

!streamlit run streamlit_app.py --server.enableCORS false --server.enableXsrfProtection false --server.port 8501 &

Visit the app in the first link, not the local link:
 NgrokTunnel: "https://16dec494fc2c.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.71.201:8501

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-08-06 18:57:45.260445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754506665.485764    7666 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754506665.549234    7666 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-06 18:57:46.090632: I tensorflow/core/platform/c